# Implementing Recommender Systems - Lab

## Introduction

In this lab, you'll practice creating a recommender system model using `surprise`. You'll also get the chance to create a more complete recommender system pipeline to obtain the top recommendations for a specific user.


## Objectives

In this lab you will: 

- Use surprise's built-in reader class to process data to work with recommender algorithms 
- Obtain a prediction for a specific user for a particular item 
- Introduce a new user with rating to a rating matrix and make recommendations for them 
- Create a function that will return the top n recommendations for a user 


For this lab, we will be using the famous 1M movie dataset. It contains a collection of user ratings for many different movies. In the last lesson, you were exposed to working with `surprise` datasets. In this lab, you will also go through the process of reading in a dataset into the `surprise` dataset format. To begin with, load the dataset into a Pandas DataFrame. Determine which columns are necessary for your recommendation system and drop any extraneous ones.

In [1]:
import pandas as pd
df = pd.read_csv('./ml-latest-small/ratings.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [3]:
# Drop unnecessary columns
new_df = df.drop(columns=['timestamp'])
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   userId   100836 non-null  int64  
 1   movieId  100836 non-null  int64  
 2   rating   100836 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 2.3 MB


It's now time to transform the dataset into something compatible with `surprise`. In order to do this, you're going to need `Reader` and `Dataset` classes. There's a method in `Dataset` specifically for loading dataframes.

In [12]:
from surprise import Reader, Dataset
# read in values as Surprise dataset 
data= Dataset.load_from_df(new_df, reader=Reader(rating_scale=(1, 5)))
data




Let's look at how many users and items we have in our dataset. If using neighborhood-based methods, this will help us determine whether or not we should perform user-user or item-item similarity

In [13]:
dataset = data.build_full_trainset()
print('Number of users: ', dataset.n_users, '\n')
print('Number of items: ', dataset.n_items)

Number of users:  610 

Number of items:  9724


## Determine the best model 

Now, compare the different models and see which ones perform best. For consistency sake, use RMSE to evaluate models. Remember to cross-validate! Can you get a model with a higher average RMSE on test data than 0.869?

In [14]:
# importing relevant libraries
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
import numpy as np

In [35]:
## Perform a gridsearch with SVD
# ⏰ This cell may take several minutes to run

param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005], 'reg_all': [0.02, 0.05]}
GS = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'])
GS.fit(data)




In [30]:
# print out optimal parameters for SVD after GridSearch
print("RMSE: ", GS.best_score['rmse'])
print("MAE: ", GS.best_score['mae'])

RMSE:  0.8796971278536636
MAE:  0.6777786899906264


In [31]:
# cross validating with KNNBasic
cross_validate(KNNBasic(), data, measures=['RMSE', 'MAE'])



Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


{'test_rmse': array([0.93681188, 0.95577591, 0.94075023, 0.94727904, 0.95148837]),
 'test_mae': array([0.71915176, 0.73318944, 0.72073039, 0.72798896, 0.72786327]),
 'fit_time': (0.060925960540771484,
  0.06899189949035645,
  0.06943273544311523,
  0.07873201370239258,
  0.07377386093139648),
 'test_time': (0.7527129650115967,
  0.6970670223236084,
  0.8300831317901611,
  0.7632288932800293,
  0.8298101425170898)}

In [32]:
# print out the average RMSE score for the test set
data_results = cross_validate(KNNBasic(), data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9532  0.9498  0.9458  0.9449  0.9498  0.9487  0.0030  
MAE (testset)     0.7298  0.7285  0.7257  0.7237  0.7283  0.7272  0.0022  
Fit time          0.06    0.08    0.07    0.07    0.07    0.07    0.00    
Test time         0.80    0.86    0.68    0.67    0.75    0.75    0.07    


In [33]:
# cross validating with KNNBaseline
cross_validate(KNNBaseline(), data, measures=['RMSE', 'MAE'])



Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


{'test_rmse': array([0.87763483, 0.86670324, 0.87610411, 0.87518609, 0.87369599]),
 'test_mae': array([0.6705043 , 0.65999773, 0.6709959 , 0.66797634, 0.6707081 ]),
 'fit_time': (0.22234702110290527,
  0.2214679718017578,
  0.22377395629882812,
  0.2005629539489746,
  0.1918630599975586),
 'test_time': (0.9598050117492676,
  1.0788938999176025,
  0.9622268676757812,
  0.8855569362640381,
  0.8773849010467529)}

In [34]:
# print out the average score for the test set
data_results = cross_validate(KNNBaseline(), data, measures=['RMSE', 'MAE'], cv=5, verbose=True)



Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBaseline on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8718  0.8720  0.8705  0.8747  0.8849  0.8748  0.0052  
MAE (testset)     0.6661  0.6690  0.6637  0.6716  0.6725  0.6686  0.0033  
Fit time          0.23    0.19    0.19    0.19    0.19    0.20    0.02    
Test time         0.92    1.05    0.90    0.95    0.88    0.94    0.06    


Based off these outputs, it seems like the best performing model is the SVD model with `n_factors = 50` and a regularization rate of 0.05. Use that model or if you found one that performs better, feel free to use that to make some predictions.

## Making Recommendations

It's important that the output for the recommendation is interpretable to people. Rather than returning the `movie_id` values, it would be far more valuable to return the actual title of the movie. As a first step, let's read in the movies to a dataframe and take a peek at what information we have about them.

In [36]:
df_movies = pd.read_csv('./ml-latest-small/movies.csv')

In [37]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## Making simple predictions
Just as a reminder, let's look at how you make a prediction for an individual user and item. First, we'll fit the SVD model we had from before.

In [38]:
svd = SVD(n_factors= 50, reg_all=0.05)
svd.fit(dataset)

In [39]:
svd.predict(2, 4)

Prediction(uid=2, iid=4, r_ui=None, est=3.1151507120356117, details={'was_impossible': False})

This prediction value is a tuple and each of the values within it can be accessed by way of indexing. Now let's put our knowledge of recommendation systems to do something interesting: making predictions for a new user!

## Obtaining User Ratings 

It's great that we have working models and everything, but wouldn't it be nice to get to recommendations specifically tailored to your preferences? That's what we'll be doing now. The first step is to create a function that allows us to pick randomly selected movies. The function should present users with a movie and ask them to rate it. If they have not seen the movie, they should be able to skip rating it. 

The function `movie_rater()` should take as parameters: 

* `movie_df`: DataFrame - a dataframe containing the movie ids, name of movie, and genres
* `num`: int - number of ratings
* `genre`: string - a specific genre from which to draw movies

The function returns:
* rating_list : list - a collection of dictionaries in the format of {'userId': int , 'movieId': int , 'rating': float}

#### This function is optional, but fun :) 

In [48]:
def movie_rater(movie_df,num, genre=None):
    pass


In [51]:
# try out the new function here!
def movie_rater(movie_df, num, genre=None):
    if genre:
        movie_df = movie_df[movie_df['genres'].str.contains(genre)]
    
    recommended_movies = movie_df.sample(n=num)
    return recommended_movies



In [55]:
# Simulate the  the movie_rater function
recommended_movies = movie_rater(df_movies, 4, genre='Comedy')
print("Recommended Movies:")
print(recommended_movies[['movieId', 'title', 'genres']])

Recommended Movies:
      movieId                            title  \
3876     5448     Hey Arnold! The Movie (2002)   
8507   114028                     Pride (2014)   
8485   113275  The Hundred-Foot Journey (2014)   
817      1077                   Sleeper (1973)   

                                   genres  
3876  Adventure|Animation|Children|Comedy  
8507                         Comedy|Drama  
8485                         Comedy|Drama  
817                         Comedy|Sci-Fi  


If you're struggling to come up with the above function, you can use this list of user ratings to complete the next segment

In [57]:
# Create a user_rating as a list of dictionaries with the following  userId, movieId, and rating
user_rating = [
	{'userId': 1000, 'movieId': 1, 'rating': 5},
	{'userId': 1000, 'movieId': 2, 'rating': 4},
	{'userId': 1000, 'movieId': 3, 'rating': 4},
	{'userId': 1000, 'movieId': 4, 'rating': 4}
]

# Display the user_rating variable
user_rating

[{'userId': 1000, 'movieId': 1, 'rating': 5},
 {'userId': 1000, 'movieId': 2, 'rating': 4},
 {'userId': 1000, 'movieId': 3, 'rating': 4},
 {'userId': 1000, 'movieId': 4, 'rating': 4}]

### Making Predictions With the New Ratings
Now that you have new ratings, you can use them to make predictions for this new user. The proper way this should work is:

* add the new ratings to the original ratings DataFrame, read into a `surprise` dataset 
* train a model using the new combined DataFrame
* make predictions for the user
* order those predictions from highest rated to lowest rated
* return the top n recommendations with the text of the actual movie (rather than just the index number) 

In [59]:
## add the new ratings to the original ratings DataFrame
new_ratings_df = pd.DataFrame(user_rating)
# Concatenate the new ratings with the original ratings DataFrame
df = pd.concat([df, new_ratings_df], ignore_index=True)
# Reload the Surprise dataset with the new ratings
data_new = Dataset.load_from_df(new_ratings_df, reader=Reader(rating_scale=(1, 5)))





In [68]:
# train a model using the new combined DataFrame
dataset_new = data_new.build_full_trainset()
svd_new = SVD(n_factors=50, reg_all=0.05)
svd_new.fit(dataset_new)



In [69]:
# make predictions for the user
# you'll probably want to create a list of tuples in the format (movie_id, predicted_score)
prediction = svd_new.predict(1000, 4)
print(f"Predicted score for user 1000 on movie 4: {prediction.est:.2f}")



Predicted rating for user 1000 on movie 4: 4.19


In [67]:
# order the predictions from highest to lowest rated
predictions = []
for movie_id in df_movies['movieId']:
    pred = svd_new.predict(1000, movie_id)
    predictions.append((movie_id, pred.est))
# Sort the predictions by estimated rating in descending order
predictions.sort(key=lambda x: x[1], reverse=True)
# Display the top 10 predictions
ranked_movies = predictions[:10]
print("Top 10 movie recommendations for user 1000:")
for movie_id, rating in ranked_movies:
    title = df_movies[df_movies['movieId'] == movie_id]['title'].values[0]
    print(f"Movie ID: {movie_id}, Title: {title}, Predicted score: {rating:.2f}")



Top 10 movie recommendations for user 1000:
Movie ID: 1, Title: Toy Story (1995), Predicted score: 4.34
Movie ID: 5, Title: Father of the Bride Part II (1995), Predicted score: 4.27
Movie ID: 6, Title: Heat (1995), Predicted score: 4.27
Movie ID: 7, Title: Sabrina (1995), Predicted score: 4.27
Movie ID: 8, Title: Tom and Huck (1995), Predicted score: 4.27
Movie ID: 9, Title: Sudden Death (1995), Predicted score: 4.27
Movie ID: 10, Title: GoldenEye (1995), Predicted score: 4.27
Movie ID: 11, Title: American President, The (1995), Predicted score: 4.27
Movie ID: 12, Title: Dracula: Dead and Loving It (1995), Predicted score: 4.27
Movie ID: 13, Title: Balto (1995), Predicted score: 4.27


 For the final component of this challenge, it could be useful to create a function `recommended_movies()` that takes in the parameters:
* `user_ratings`: list - list of tuples formulated as (user_id, movie_id) (should be in order of best to worst for this individual)
* `movie_title_df`: DataFrame 
* `n`: int - number of recommended movies 

The function should use a `for` loop to print out each recommended *n* movies in order from best to worst

In [ ]:
# return the top n recommendations using the 
def recommended_movies(user_ratings,movie_title_df,n):
        pass
            
recommended_movies(ranked_movies,df_movies,5)

## Level Up (Optional)

* Try and chain all of the steps together into one function that asks users for ratings for a certain number of movies, then all of the above steps are performed to return the top $n$ recommendations
* Make a recommender system that only returns items that come from a specified genre

## Summary

In this lab, you got the chance to implement a collaborative filtering model as well as retrieve recommendations from that model. You also got the opportunity to add your own recommendations to the system to get new recommendations for yourself! Next, you will learn how to use Spark to make recommender systems.